# Загрузка данных

In [ ]:
import pandas as pd
df = pd.read_csv('../datasets/kaggle_survey_2022_responses.csv')
name2question = df.iloc[0]
df.head (5)


# 1
Узнай самый часто встречаемый возраст респондентов

In [ ]:
task1 = df[['Q2']] # Присваиваем переменной значение столбца Q2
task1 = task1.drop(0) # Удаляем строку с вопросом
task1 = task1['Q2'].value_counts() # подсчитываем частоту встречаемости каждого возраста в столбце Q2
age = task1.idxmax() # Находим возрастную группу
count = task1.max() # Находим количество вхождений
f'Возрастная группа {age}, количество повторений {count}'

Cамый часто встречаемый возраст 18-21, количество повторений 4559.

# 2
Посчитай долю женщин среди русских участников опроса, которые уже не студенты.

In [ ]:
total = df.query('Q4 == "Russia" and Q5 == "No"')
total = total[['Q3', 'Q4', 'Q5']]
total['Q3'].value_counts(dropna = False)
woman = 0
man = 0
for index, value in total["Q3"].items():
    if value == 'Woman':
        woman += 1
    elif value == 'Man':
        man += 1
task2 = round (woman / (man + woman), 2)
f'Доля женщин = {task2}'

Доля женщин = 0.24

# 3
Найди среднее время заполнения опроса в **минутах**.

In [ ]:
time = df [['Duration (in seconds)']].drop(0)
time ['Duration (in seconds)'] = time['Duration (in seconds)'].astype(int)
time = time ['Duration (in seconds)'].mean()
task3 = round ((time / 60), 2)
f'Среднее время заполнения опроса {task3} минут.'

Среднее время заполнения опроса 168.17 минут.

# 4
Выведи топ-3 образовательных курса, которые указали респонденты.

In [ ]:
total = {}
q6_columns = [col for col in df.columns if col.startswith('Q6')] # Создаем список по столбцам Q6
courses = df[q6_columns] # Присваиваем переменной датафрейм по списку Q6
courses = courses.drop(0) # Удаляем строку с вопросом
courses = courses[q6_columns].melt(var_name='TOP', value_vars=q6_columns) # Объединяем столбцы в одну таблицу
task4 = courses['value'].value_counts().iloc[:3] # Выводим 3 топ курса
task4


- Coursera                                                 9699
- University Courses (resulting in a university degree)    6780
- Kaggle Learn Courses                                     6628

# 5
Узнай топ-5 языков программирования, которые регулярно используют Data Scientists в своей работе

In [ ]:
# Присваиваем вопрос переменной
question = 'What programming languages do you use on a regular basis?'
# str.contains() для фильтрации столбцов, содержащих текст вопроса question
# Метод возвращает логический массив, который используется для фильтрации столбцов в объекте df.columns
filtered_cols = df.columns[df.iloc[0].str.contains(question)]
# Метод get_loc() объекта df.columns, который находит индекс столбца, содержащего текст вопроса. 
col_idx = df.columns.get_loc(filtered_cols[0])
# Так определил с какого столбца начинаются вопросы про язык программирования Q12_1
(df.iloc[:, col_idx])
# df.loc - это метод для доступа и изменения элементов в объекте DataFrame. Он используется для выбора данных на основе меток строк или столбцов.
language_DS = df.loc[:, ['Q23'] + ['Q12_' + str(i) for i in range(1, 16)]]
# Удаляем строку с вопросом
language_DS = language_DS.drop(0)
# Фильтруем по названию специальности
language_DS = language_DS.query( 'Q23 == "Data Scientist"')
# Создаем список по столбцам Q12
q12_columns = [col for col in language_DS.columns if col.startswith('Q12')]
# Объединяем столбцы в одну таблицу
total_lg_DS = language_DS.melt(id_vars=['Q23'], value_vars = q12_columns, var_name='column', value_name='language')
# Удаляю столбец с именами столбцов
total_lg_DS = total_lg_DS.drop(columns=['column'])
# Выводим TOP 5 языков программирования
task5 = total_lg_DS['language'].value_counts().iloc[:5]
task5


- Python        1808
- SQL           1140
- R              530
- Bash           248
- Javascript     187

# 6
Узнай средний доход среди 'Data Scientist' специалистов и средний доход среди 'Machine Learning/ MLops Engineer' \
Т.к. в колонке Q29 указан промежуток, то возьми среднее значение из промежутка. 
Средние доход **округли до целого.**

In [ ]:
employees_mapping = {}
q29 = df ['Q29']
q29 = q29.drop (0)
for index, value in q29.items():
   if (type (value)) == str:
      if value == '>$1,000,000':
         range_start = 1000000
         employees_mapping[value] = range_start
      else:
         range_split = value.split('-')
         if len (range_split) == 2:
            range_start = int(range_split[0].replace(',', '').strip('$'))
            range_end = int(range_split[1].replace(',', ''))
            range_mean = round((range_start + range_end) / 2)
            employees_mapping[value] = range_mean

job_and_money = df[['Q23', 'Q29']]
job_and_money = job_and_money.drop(0)

New_money = job_and_money['Q29-NEW'] = job_and_money['Q29'].map(employees_mapping)
filter_DS = job_and_money.query( 'Q23 == "Data Scientist"')
DS_money = filter_DS[['Q23','Q29-NEW']].dropna()
task6_DS = DS_money.groupby(['Q23']).mean().sort_values('Q29-NEW', ascending=False)
money_DS = task6_DS.iloc[0]['Q29-NEW']
money_DS = round(money_DS)
job_DS = task6_DS.index[0]

filter_ML = job_and_money.query( 'Q23 == "Machine Learning/ MLops Engineer"')
ML_money = filter_ML[['Q23','Q29-NEW']].dropna()
task6_ML = ML_money.groupby(['Q23']).mean().sort_values('Q29-NEW', ascending=False)
money_ML = task6_ML.iloc[0]['Q29-NEW']
money_ML = round(money_ML)
job_ML = task6_ML.index[0]

f'Средни доход {job_DS} = {money_DS}  Средни доход {job_ML} = {money_ML}'


- Средни доход Data Scientist = 70337
- Средни доход Machine Learning/ MLops Engineer = 67438